In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

In [2]:
df = pd.read_csv("Bookings.csv")

# Data Exploration

In [3]:
df.head()

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,V_TAT,C_TAT,...,Canceled_Rides_by_Driver,Incomplete_Rides,Incomplete_Rides_Reason,Booking_Value,Payment_Method,Ride_Distance,Driver_Ratings,Customer_Rating,Vehicle Images,Unnamed: 20
0,2024-07-26 14:00:00,14:00:00,CNR7153255142,Canceled by Driver,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,...,Personal & Car related issue,NaN,NaN,444,NaN,0,NaN,NaN,#NAME?,NaN
1,2024-07-25 22:20:00,22:20:00,CNR2940424040,Success,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,...,NaN,No,NaN,158,Cash,13,4.1,4.0,#NAME?,NaN
2,2024-07-30 19:59:00,19:59:00,CNR2982357879,Success,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,...,NaN,No,NaN,386,UPI,40,4.2,4.8,#NAME?,NaN
3,2024-07-22 3:15:00,03:15:00,CNR2395710036,Canceled by Customer,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,...,NaN,NaN,NaN,384,NaN,0,NaN,NaN,#NAME?,NaN
4,2024-07-02 9:02:00,09:02:00,CNR1797421769,Success,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,...,NaN,No,NaN,822,Credit Card,45,4.0,3.0,#NAME?,NaN


In [4]:
df.shape

(103024, 21)

- V_TAT - Vehicle turnaround time (Time vehicle takes to pickup customer after accepting the ride.)
- C_TAT - Customer turn around time (Time taken by customer to board cab after cab is reached at location.)


In [5]:
df.describe()

,V_TAT,C_TAT,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating,Unnamed: 20
count,63967.000000,63967.000000,103024.000000,103024.000000,63967.000000,63967.000000,0.0
mean,170.876952,84.873372,548.751883,14.189927,3.997457,3.998313,NaN
std,80.803640,36.005100,536.541221,15.776270,0.576834,0.578957,NaN
min,35.000000,25.000000,100.000000,0.000000,3.000000,3.000000,NaN
25%,98.000000,55.000000,242.000000,0.000000,3.500000,3.500000,NaN
50%,168.000000,85.000000,386.000000,8.000000,4.000000,4.000000,NaN
75%,238.000000,115.000000,621.000000,26.000000,4.500000,4.500000,NaN
max,308.000000,145.000000,2999.000000,49.000000,5.000000,5.000000,NaN


- V_TAT = mean = 50%val (distribution is normal)
- C_TAT = mean = 50%val (distribution is normal)
- Booking_value = Has some outliers.
- Ride_distance = Has some outliers.
- Driver_ratings = Almost driver ratings are same.
- Customer_ratings = Almost customer ratings are same. 
#### Numerical colums (Above metioned has almost no effect on bookings.)

In [6]:
# count missing values in each column
df.isnull().sum().reset_index()
null_percent = (df.isnull().sum() / len(df)) * 100
null_percent

Date                            0.000000
Time                            0.000000
Booking_ID                      0.000000
Booking_Status                  0.000000
Customer_ID                     0.000000
Vehicle_Type                    0.000000
Pickup_Location                 0.000000
Drop_Location                   0.000000
V_TAT                          37.910584
C_TAT                          37.910584
Canceled_Rides_by_Customer     89.809171
Canceled_Rides_by_Driver       82.107082
Incomplete_Rides               37.910584
Incomplete_Rides_Reason        96.189237
Booking_Value                   0.000000
Payment_Method                 37.910584
Ride_Distance                   0.000000
Driver_Ratings                 37.910584
Customer_Rating                37.910584
Vehicle Images                  0.000000
Unnamed: 20                   100.000000
dtype: float64

## DATA PREPROCESSING


### Cleaning Opportunities 
1. Filling null values in V_TAT and C_TAT column 
2. Date column contains Date + Time and datatype is object.
3. Null values in driver rating columns.
4. Customer rating column null values. 
5. Vehicle images and Unnamed column is not needed.
6. Time columns values are in object format.


In [7]:
# Removing vehicle images and unnamed column 
df.drop(columns={"Vehicle Images","Unnamed: 20"}, inplace=True)

1. Total 10499 rides were cancelled by customer
2. Total 18434 rides were cancelled by drivers 
3. Total 10124 rides were not executed due to other reasons. 

In [8]:
df['Date'] = df['Date'].str.split(' ').str[0]

In [9]:
df['Date'] = pd.to_datetime(df['Date'])

In [10]:
# lets replace booking status column with succes = 1 and cancelled by drivers/riders = 0
df['Booking_Status']=df['Booking_Status'].replace({'Success':1,'Canceled by Driver':0,'Canceled by Customer':0})
df

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,V_TAT,C_TAT,Canceled_Rides_by_Customer,Canceled_Rides_by_Driver,Incomplete_Rides,Incomplete_Rides_Reason,Booking_Value,Payment_Method,Ride_Distance,Driver_Ratings,Customer_Rating
0,2024-07-26,14:00:00,CNR7153255142,0,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,NaN,Personal & Car related issue,NaN,NaN,444,NaN,0,NaN,NaN
1,2024-07-25,22:20:00,CNR2940424040,1,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,NaN,NaN,No,NaN,158,Cash,13,4.1,4.0
2,2024-07-30,19:59:00,CNR2982357879,1,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,NaN,NaN,No,NaN,386,UPI,40,4.2,4.8
3,2024-07-22,03:15:00,CNR2395710036,0,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,Driver is not moving towards pickup location,NaN,NaN,NaN,384,NaN,0,NaN,NaN
4,2024-07-02,09:02:00,CNR1797421769,1,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,NaN,NaN,No,NaN,822,Credit Card,45,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103019,2024-07-31,09:06:00,CNR9488489435,1,CID371654,Prime Plus,Richmond Town,Varthur,245.0,35.0,NaN,NaN,No,NaN,111,Cash,41,3.6,3.8
103020,2024-07-31,15:12:00,CNR3151743100,1,CID334158,Auto,Vijayanagar,Richmond Town,84.0,145.0,NaN,NaN,No,NaN,1097,UPI,17,4.3,3.3
103021,2024-07-31,13:59:00,CNR1286151233,1,CID113188,Prime SUV,Bannerghatta Road,JP Nagar,35.0,75.0,NaN,NaN,No,NaN,2201,Cash,37,3.6,3.2
103022,2024-07-31,14:56:00,CNR2027162035,1,CID118301,eBike,Indiranagar,Magadi Road,210.0,140.0,NaN,NaN,No,NaN,267,UPI,47,3.4,3.1


In [11]:
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')


In [27]:
new_df=df[['Date','Time','Booking_ID','Booking_Status','Customer_ID','Vehicle_Type','Pickup_Location','Drop_Location','V_TAT','C_TAT','Canceled_Rides_by_Customer','Canceled_Rides_by_Driver','Booking_Value','Payment_Method','Ride_Distance','Driver_Ratings','Customer_Rating']]
new_df

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,V_TAT,C_TAT,Canceled_Rides_by_Customer,Canceled_Rides_by_Driver,Booking_Value,Payment_Method,Ride_Distance,Driver_Ratings,Customer_Rating
0,2024-07-26,1900-01-01 14:00:00,CNR7153255142,0,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,NaN,Personal & Car related issue,444,NaN,0,NaN,NaN
1,2024-07-25,1900-01-01 22:20:00,CNR2940424040,1,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,NaN,NaN,158,Cash,13,4.1,4.0
2,2024-07-30,1900-01-01 19:59:00,CNR2982357879,1,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,NaN,NaN,386,UPI,40,4.2,4.8
3,2024-07-22,1900-01-01 03:15:00,CNR2395710036,0,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,Driver is not moving towards pickup location,NaN,384,NaN,0,NaN,NaN
4,2024-07-02,1900-01-01 09:02:00,CNR1797421769,1,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,NaN,NaN,822,Credit Card,45,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103019,2024-07-31,1900-01-01 09:06:00,CNR9488489435,1,CID371654,Prime Plus,Richmond Town,Varthur,245.0,35.0,NaN,NaN,111,Cash,41,3.6,3.8
103020,2024-07-31,1900-01-01 15:12:00,CNR3151743100,1,CID334158,Auto,Vijayanagar,Richmond Town,84.0,145.0,NaN,NaN,1097,UPI,17,4.3,3.3
103021,2024-07-31,1900-01-01 13:59:00,CNR1286151233,1,CID113188,Prime SUV,Bannerghatta Road,JP Nagar,35.0,75.0,NaN,NaN,2201,Cash,37,3.6,3.2
103022,2024-07-31,1900-01-01 14:56:00,CNR2027162035,1,CID118301,eBike,Indiranagar,Magadi Road,210.0,140.0,NaN,NaN,267,UPI,47,3.4,3.1


- cancelled_rides_by_customers = cancelled = 1 , not_cancelled = 0
- cancelled_rides_by_drivers = cancelled = 1 , not_cancelled = 0
- Payment_method = replace NaN values with 0 i.e. no payment done.
- Driver ratings - replae Nan values with 0. (Data is showing almost normal distribution)
- Customer ratings - replace NaN values with 0. (Data is showing almost normal distribution.)


In [14]:
# Cancelled_rides column replacement
new_df['Canceled_Rides_by_Customer'] = new_df['Canceled_Rides_by_Customer'].notna().astype(int)
new_df['Canceled_Rides_by_Driver'] = new_df['Canceled_Rides_by_Driver'].notna().astype(int)

C:\Users\Dell\AppData\Local\Temp\ipykernel_13180\1540107749.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Canceled_Rides_by_Customer'] = new_df['Canceled_Rides_by_Customer'].notna().astype(int)
C:\Users\Dell\AppData\Local\Temp\ipykernel_13180\1540107749.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Canceled_Rides_by_Driver'] = new_df['Canceled_Rides_by_Driver'].notna().astype(int)


In [15]:
new_df

,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,V_TAT,C_TAT,Canceled_Rides_by_Customer,Canceled_Rides_by_Driver,Booking_Value,Payment_Method,Ride_Distance,Driver_Ratings,Customer_Rating
0,2024-07-26,1900-01-01 14:00:00,CNR7153255142,0,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,0,1,444,NaN,0,NaN,NaN
1,2024-07-25,1900-01-01 22:20:00,CNR2940424040,1,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,0,0,158,Cash,13,4.1,4.0
2,2024-07-30,1900-01-01 19:59:00,CNR2982357879,1,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,0,0,386,UPI,40,4.2,4.8
3,2024-07-22,1900-01-01 03:15:00,CNR2395710036,0,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,1,0,384,NaN,0,NaN,NaN
4,2024-07-02,1900-01-01 09:02:00,CNR1797421769,1,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,0,0,822,Credit Card,45,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103019,2024-07-31,1900-01-01 09:06:00,CNR9488489435,1,CID371654,Prime Plus,Richmond Town,Varthur,245.0,35.0,0,0,111,Cash,41,3.6,3.8
103020,2024-07-31,1900-01-01 15:12:00,CNR3151743100,1,CID334158,Auto,Vijayanagar,Richmond Town,84.0,145.0,0,0,1097,UPI,17,4.3,3.3
103021,2024-07-31,1900-01-01 13:59:00,CNR1286151233,1,CID113188,Prime SUV,Bannerghatta Road,JP Nagar,35.0,75.0,0,0,2201,Cash,37,3.6,3.2
103022,2024-07-31,1900-01-01 14:56:00,CNR2027162035,1,CID118301,eBike,Indiranagar,Magadi Road,210.0,140.0,0,0,267,UPI,47,3.4,3.1


In [16]:
# Driver and customer ratings column fill with 0 for NaN
new_df['Driver_Ratings'] = new_df['Driver_Ratings'].fillna(0)
new_df['Customer_Rating'] = new_df['Customer_Rating'].fillna(0)
new_df


C:\Users\Dell\AppData\Local\Temp\ipykernel_13180\3106024483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Driver_Ratings'] = new_df['Driver_Ratings'].fillna(0)
C:\Users\Dell\AppData\Local\Temp\ipykernel_13180\3106024483.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Customer_Rating'] = new_df['Customer_Rating'].fillna(0)


,Date,Time,Booking_ID,Booking_Status,Customer_ID,Vehicle_Type,Pickup_Location,Drop_Location,V_TAT,C_TAT,Canceled_Rides_by_Customer,Canceled_Rides_by_Driver,Booking_Value,Payment_Method,Ride_Distance,Driver_Ratings,Customer_Rating
0,2024-07-26,1900-01-01 14:00:00,CNR7153255142,0,CID713523,Prime Sedan,Tumkur Road,RT Nagar,NaN,NaN,0,1,444,NaN,0,0.0,0.0
1,2024-07-25,1900-01-01 22:20:00,CNR2940424040,1,CID225428,Bike,Magadi Road,Varthur,203.0,30.0,0,0,158,Cash,13,4.1,4.0
2,2024-07-30,1900-01-01 19:59:00,CNR2982357879,1,CID270156,Prime SUV,Sahakar Nagar,Varthur,238.0,130.0,0,0,386,UPI,40,4.2,4.8
3,2024-07-22,1900-01-01 03:15:00,CNR2395710036,0,CID581320,eBike,HSR Layout,Vijayanagar,NaN,NaN,1,0,384,NaN,0,0.0,0.0
4,2024-07-02,1900-01-01 09:02:00,CNR1797421769,1,CID939555,Mini,Rajajinagar,Chamarajpet,252.0,80.0,0,0,822,Credit Card,45,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103019,2024-07-31,1900-01-01 09:06:00,CNR9488489435,1,CID371654,Prime Plus,Richmond Town,Varthur,245.0,35.0,0,0,111,Cash,41,3.6,3.8
103020,2024-07-31,1900-01-01 15:12:00,CNR3151743100,1,CID334158,Auto,Vijayanagar,Richmond Town,84.0,145.0,0,0,1097,UPI,17,4.3,3.3
103021,2024-07-31,1900-01-01 13:59:00,CNR1286151233,1,CID113188,Prime SUV,Bannerghatta Road,JP Nagar,35.0,75.0,0,0,2201,Cash,37,3.6,3.2
103022,2024-07-31,1900-01-01 14:56:00,CNR2027162035,1,CID118301,eBike,Indiranagar,Magadi Road,210.0,140.0,0,0,267,UPI,47,3.4,3.1


In [28]:
new_df.describe()

,Date,Time,V_TAT,C_TAT,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating
count,103024,103024,63967.000000,63967.000000,103024.000000,103024.000000,63967.000000,63967.000000
mean,2024-07-15 23:33:01.425687040,1900-01-01 11:58:37.453991424,170.876952,84.873372,548.751883,14.189927,3.997457,3.998313
min,2024-07-01 00:00:00,1900-01-01 00:00:00,35.000000,25.000000,100.000000,0.000000,3.000000,3.000000
25%,2024-07-08 00:00:00,1900-01-01 06:01:00,98.000000,55.000000,242.000000,0.000000,3.500000,3.500000
50%,2024-07-16 00:00:00,1900-01-01 11:58:00,168.000000,85.000000,386.000000,8.000000,4.000000,4.000000
75%,2024-07-24 00:00:00,1900-01-01 17:57:00,238.000000,115.000000,621.000000,26.000000,4.500000,4.500000
max,2024-07-31 00:00:00,1900-01-01 23:59:00,308.000000,145.000000,2999.000000,49.000000,5.000000,5.000000
std,NaN,NaN,80.803640,36.005100,536.541221,15.776270,0.576834,0.578957


# Univariate Analysis

In [31]:
# Seperating numerical and categorical features
numerical_col = new_df[["V_TAT","C_TAT","Canceled_Rides_by_Customer","Canceled_Rides_by_Driver","Booking_Value","Ride_Distance","Driver_Ratings","Customer_Rating"]]
categorical_col = new_df[['Vehicle_Type','Pickup_Location']]

In [32]:
numerical_col.describe()

,V_TAT,C_TAT,Booking_Value,Ride_Distance,Driver_Ratings,Customer_Rating
count,63967.000000,63967.000000,103024.000000,103024.000000,63967.000000,63967.000000
mean,170.876952,84.873372,548.751883,14.189927,3.997457,3.998313
std,80.803640,36.005100,536.541221,15.776270,0.576834,0.578957
min,35.000000,25.000000,100.000000,0.000000,3.000000,3.000000
25%,98.000000,55.000000,242.000000,0.000000,3.500000,3.500000
50%,168.000000,85.000000,386.000000,8.000000,4.000000,4.000000
75%,238.000000,115.000000,621.000000,26.000000,4.500000,4.500000
max,308.000000,145.000000,2999.000000,49.000000,5.000000,5.000000
